In [1]:
import pandas as pd
import numpy as np

In [3]:
messi = pd.read_csv('house_prices_train.csv')

In [4]:
messi['Erf Size'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [5]:
ronaldo = pd.read_csv('house_prices_test.csv')

In [6]:
ronaldo['Erf Size'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [7]:
#from geopandas.tools.geocoding import reverse_geocoder 

In [8]:
digits_col = ronaldo.select_dtypes(include=(np.int64,float), exclude=object).columns

In [9]:
caterg_col = ronaldo.select_dtypes(exclude=(np.int64,float), include=object).columns

In [10]:
def missing(df):
    missing_values = round(df.isna().sum()/len(df) * 100,2)
    df = missing_values.to_frame(name='Missing')
    count = df[df['Missing']> 0]['Missing'].count()
    return df[['Missing']].sort_values(by ='Missing',ascending=False).head(count)


In [11]:
missing(messi)

,Missing
Garages,39.02
Erf Size,21.82
Bathrooms,13.82
Bedrooms,2.85
Price,0.27


In [12]:
messi['Price'] = messi['Price'].fillna(messi['Price'].median())

In [13]:
missing(messi)

,Missing
Garages,39.02
Erf Size,21.82
Bathrooms,13.82
Bedrooms,2.85


In [14]:
def feature_transformations(df):

    # drop nulls
    df['Garages'] = df['Garages'].fillna('0')
    df['Bedrooms'] = df['Bedrooms'].fillna('1')
    
    df['Erf Size'] = df['Erf Size'].fillna(df['Erf Size'].median())
    df['Bathrooms'] = df['Bathrooms'].fillna('1')

    return df
messi = feature_transformations(messi)
ronaldo = feature_transformations(ronaldo)

In [15]:
missing(ronaldo)

,Missing


In [16]:
missing(messi)

,Missing


In [17]:
X_messi = messi.drop(['Price'] ,axis = 1 )

In [18]:
X_messi = messi.drop(['Price'] ,axis = 1 )

In [19]:
X_ronaldo = ronaldo

In [20]:
y_messi = messi['Price']

In [21]:
messi.head()

,ID,Bathrooms,Bedrooms,Erf Size,Garages,Listing Date,Price,Type of Property,Location
0,1003,4,4,230,2,2019-10-28,4299000.0,House,"lat: 51.5073509,long: -0.1277583"
1,1011,2,3,1733,2,2019-06-05,2800000.0,House,"lat: 51.5073509,long: -0.1277583"
2,1016,2,3,987,1,2019-10-31,980000.0,House,"lat: 51.5073509,long: -0.1277583"
3,1019,2,3,1539,2,2019-10-14,1795000.0,House,"lat: 51.5073509,long: -0.1277583"
4,1029,2,3,70,0,2019-10-02,837780.0,House,"lat: 51.5073509,long: -0.1277583"


In [22]:
def encode_dummies(barcelona):
    df_dummies = pd.get_dummies(barcelona,  columns = caterg_col , drop_first = True)
    return df_dummies

In [23]:
X_liga = encode_dummies(pd.concat([X_messi,X_ronaldo], axis=0))

In [24]:
X_messi = X_liga[:round(len(X_messi))]

In [25]:
X_ronaldo = X_liga[round(len(X_messi)):]

In [26]:
y_messi = np.log(y_messi)

In [27]:
from sklearn.linear_model import Lasso

In [28]:
lm = Lasso(alpha = 0.01,normalize=True)

lm.fit(X_messi, y_messi)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=True, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [29]:
# check training accuracy
train_lm = lm.predict(X_messi)

In [30]:
from sklearn import metrics

In [31]:
#linear training
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_messi, train_lm))
rmse

0.568752486848341

In [32]:
#linear testing rmse
test_lm =np.exp(lm.predict(X_ronaldo))

In [33]:
from sklearn.ensemble import RandomForestRegressor

In [34]:
forest_reg = RandomForestRegressor(random_state=0)

In [35]:
forest_reg.fit(X_messi,y_messi)

C:\Users\user\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [36]:
ff = (forest_reg.predict(X_messi))

In [37]:
print('Score:', metrics.mean_squared_error(y_messi,ff)) 

Score: 0.02843835411091605


In [38]:
y_pred = np.exp(forest_reg.predict(X_ronaldo))

In [39]:
#grid_param - 

In [40]:
from sklearn import ensemble


In [41]:
from sklearn.ensemble import GradientBoostingRegressor

In [42]:
model = ensemble.GradientBoostingRegressor()

In [43]:
model.fit(X_messi,y_messi)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [44]:
ffz = (model.predict(X_messi))

In [45]:
print('Score:', metrics.mean_squared_error(y_messi,ffz)) 

Score: 0.09807976143794467


In [46]:
z_pred = np.exp(model.predict(X_ronaldo))

In [47]:
from sklearn.linear_model import Ridge

In [48]:
Ridge = Ridge(alpha=0.009,normalize=True)

In [49]:
Ridge.fit(X_messi, y_messi)

Ridge(alpha=0.009, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=True, random_state=None, solver='auto', tol=0.001)

In [50]:
from sklearn import metrics

In [51]:
messi_ridge = (Ridge.predict(X_messi))

In [52]:
print('Score:', metrics.mean_squared_error(y_messi, messi_ridge)) 

Score: 0.04221910067486557


In [53]:
ronaldo_ridge = np.exp(Ridge.predict(X_ronaldo))

In [ ]:
ridge =pd.DataFrame({'ID': ronaldo["ID"], 'Price': y_pred})
ridge.to_csv('ridge_prediction822.csv', index=False)